In [222]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import math
from nltk import word_tokenize,sent_tokenize
import string

In [223]:
from ipynb.fs.full.TFIDF import *
stop = stopwords.words("english")
stop = stop + list(string.punctuation)

In [224]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []
    for sentence in article:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    return sentences


In [225]:
def cos_distance(a, b):
    if len(a) != len(b):
        return False
    numerator = 0
    denoma = 0
    denomb = 0
    for i in range(len(a)):
        numerator += a[i]*b[i]
        denoma += abs(a[i])**2
        denomb += abs(b[i])**2
    result = 1 - numerator / (math.sqrt(denoma)*math.sqrt(denomb))
    return result

In [226]:
def sentence_similarity(idx1, idx2,sent,values,stopwords=None):
    if stopwords is None:
        stopwords = []
    sent1=sent[idx1]
    sent2=sent[idx2]
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += values[idx1][w]
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += values[idx2][w]
 
    return 1 - cos_distance(vector1, vector2)


In [227]:
def build_similarity_matrix(clean_text,values, stop_words):
    # Create an empty similarity matrix
    #print(clean_text)
    similarity_matrix = np.zeros((len(clean_text), len(clean_text)))
    for idx1 in range(len(clean_text)):
        for idx2 in range(len(clean_text)):
            if idx1 == idx2:
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(idx1, idx2,clean_text,values,stop_words)

    return similarity_matrix


In [228]:
def pagerank(graph):
    n=graph.shape[0]
    A = np.zeros((n,1))
    A.fill(1/n)
    for _ in range(1,100):
        A=np.matmul(graph,A)
    return A

In [229]:
def get_text(text):
    arr = sent_tokenize(text)
    new_arr=[]
    total_words=[]
    for w in arr:
        words = word_tokenize(w)
        clean=[]
        for x in words:
            if x.lower() not in stop:
                clean.append(x.lower())
                total_words.append(x.lower())
        new_arr.append(clean)
    return new_arr

In [235]:
def summary(file_name):
    f = open("summarize.txt", "r")
    stop_words = stopwords.words('english')
    text = f.read()
    sentences=sent_tokenize(text)
    clean_text = get_text(text)
    
    values=get_TF_IDF(file_name)
    top_n=3
    sentence_similarity_martix = build_similarity_matrix(clean_text,values, stop_words)
    scores = pagerank(sentence_similarity_martix)
    ranked_sentence=[]
    for i in range(len(sentences)):
        ranked_sentence.append((scores[i],sentences[i]))
    Output = sorted(ranked_sentence, key = lambda x: float(x[0]),reverse=True) 
    summarize_text="";
    for i in range(top_n):
        summarize_text+=" "+Output[i][1]
    print(summarize_text)
    return summarize_text

In [236]:
answer = summary("summarize.txt")
f = open("answer.txt", "w")
f.write(answer)
f.close()

 [46] Besides school cricket, he also played club cricket, initially representing John Bright Cricket Club in Bombay's premier club cricket tournament, the Kanga League,[43] and later went on to play for the Cricket Club of India. [49] A couple of months later, former Indian batsman Sunil Gavaskar gave him a pair of his own ultra light pads and consoled him to not get disheartened for not getting the Bombay Cricket Association's "Best junior cricket award" (He was 14 years that time). [48] On 20 January 1987, he also turned out as substitute for Imran Khan's side in an exhibition game at Brabourne Stadium in Bombay, to mark the golden jubilee of Cricket Club of India.
